In [2]:
TOKEN = 'ODA3Mzc5MjYzMjc1MTM5MTEy.YB3IiA.3_cCFh5lAaQ5njle2DC4kVbRYy0'
BOT_PREFIX = '<'
OWNER_ID = 341892286189404162

In [ ]:
from google.colab import drive
 
drive.mount("/content/gdrive")
import os
sample_dir = 'results'
os.makedirs(sample_dir, exist_ok=True)
sample_dir_2 = 'Pokemon'
os.makedirs(sample_dir_2, exist_ok=True)
sample_dir_3 = 'Autism'
os.makedirs(sample_dir_3, exist_ok=True)
sample_dir_4 = 'out'
os.makedirs(sample_dir_4, exist_ok=True)

In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install discord.py==1.4.2
!pip install nest_asyncio==1.4.0
!pip install face-recognition==1.2.3
!pip3 install transformers==3.5.0

In [ ]:
!git clone https://github.com/Jahysama/stylegan2_pytorch
!git clone https://github.com/Jahysama/stylegan_forpokemon
!git clone https://github.com/Jahysama/stylegan2-ada-pytorch
#!git clone  https://github.com/sberbank-ai/ru-gpts

In [6]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torchvision
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.utils import save_image
import random
import numpy as np
import requests
from io import BytesIO
from random import randint
import torch.nn.functional as F
import face_recognition
from PIL import Image, ImageOps
import json
#from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline, set_seed
import discord
import asyncio
import nest_asyncio
from discord.ext import commands
import os
#import pandas as pd

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [8]:
def convert_to_int(string):
  out = int.from_bytes(bytes(string, 'UTF-8'), byteorder='big')%947
  return int(out)

def convert_to_int_2(string):
  out = int.from_bytes(bytes(string, 'UTF-8'), byteorder='big')%1002583
  return int(out)

In [9]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
 
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)
 
    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [10]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]
def save_sample(file_name, latent_tensors, generator, sample_dir):
    fake_images = generator(latent_tensors)[0]
    save_image(denorm(fake_images), os.path.join(sample_dir, file_name))

In [ ]:
class DensNet(nn.Module):
    def __init__(self, num_classes=2, num_channels=3):
        super().__init__()
        preloaded = torchvision.models.densenet161(pretrained=True)
        self.features = preloaded.features
        self.features.conv0 = nn.Conv2d(num_channels, 96, 7, 2, 3)
        self.classifier = nn.Linear(2208, num_classes, bias=True)
        self.bn = nn.BatchNorm1d(2208)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_max_pool2d(out, (1, 1)).view(features.size(0), -1)
        #out = self.bn(out)
        out = self.classifier(out)
        return out

def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    classes = ['autistic', 'non autistic']
    return classes[preds[0].item()]

Net = DensNet()
Net.eval()

In [ ]:
Net.load_state_dict(torch.load('/content/gdrive/MyDrive/Data/autism_epoch-34.pt'))

In [ ]:
device = get_default_device()
Net = to_device(Net, device = device)
device

In [ ]:
!python /content/stylegan2_pytorch/run_convert_from_tf.py --input=/content/gdrive/MyDrive/Data/2020-01-11-skylion-stylegan2-animeportraits-networksnapshot-024664.pkl --output checkpoint
!python /content/stylegan2_pytorch/run_convert_from_tf.py --input=/content/gdrive/MyDrive/Data/2020-05-06-arfa-stylegan2-e621-r-512-3194880.pkl --output checkpoint2
!python /content/stylegan2_pytorch/run_convert_from_tf.py --input=/content/gdrive/MyDrive/Data/stylegan2-cat-config-f.pkl --output checkpoint3
!python /content/stylegan2_pytorch/run_convert_from_tf.py --input=/content/gdrive/MyDrive/Data/ffhq-512-avg-tpurun1.pkl --output checkpoint4
!python /content/stylegan2_pytorch/run_convert_from_tf.py --input=/content/gdrive/MyDrive/Data/TDPDNE_702464.pkl --output checkpoint5

In [ ]:
  nest_asyncio.apply()

  bot = commands.Bot(command_prefix=BOT_PREFIX, help_command=None)
  generator = pipeline('text-generation', model='gpt2')


 
  @bot.event
  async def on_ready():

     print('Ready')
     global BOT_PREFIX
     activity = discord.Game(name=BOT_PREFIX+"help")
     await bot.change_presence(status=discord.Status.idle, activity=activity)
 
  @bot.command(name="fuuka",pass_context = True , aliases=['f'])
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def fuuka(ctx):
    if not in_ban(ctx.message.author):
      num = randint(0, 50)
      path = '/content/gdrive/MyDrive/Data/FuukaPic/{0}.jpg'.format(str(num))
      await ctx.send(file=discord.File(path))
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)


  @bot.event
  async def on_message(message):
      global human
      global botu
      global full

      mention = f'<@!{bot.user.id}>'
      if (((mention in message.content) or f'<@{bot.user.id}>' in message.content) and message.author.id != bot.user.id):
        channel = message.channel
        if f'<@{bot.user.id}>' in message.content:
          txtinput = message.content.replace(f'<@{bot.user.id}>', "")
        else:
          txtinput = message.content.replace(f"<@!{bot.user.id}>", "")
        input = "Boy said, " + txtinput + "\nGirl said, "
        ans = ""
        input_len = len(input)
        generated = generator(input, max_length=input_len + 50, num_return_sequences=1)
        ans = generated[0]['generated_text'].splitlines()
        ans = str(ans[1])
        ans = ans.replace("Girl said,","")
        await channel.send(f'<@!{message.author.id}>' +" " +ans)

      await bot.process_commands(message)


 
  @bot.command(pass_context = True , aliases=['h'])
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def help(ctx):
    global BOT_PREFIX
    if not in_ban(ctx.message.author):
 
        embed = discord.Embed(
            colour=discord.Colour.from_rgb(153, 102, 51),
            title="Fuuka",
            description="Prefix: " +BOT_PREFIX
        )
 
        #embed.set_author(name="Author", icon_url="https://cdn.discordapp.com/attachments/755435067282555011/781253419486150666/fuuka.png")
        #embed.set_image(url="https://cdn.discordapp.com/attachments/755435067282555011/781254299908702208/puuka.png")
        embed.set_thumbnail(url="https://cdn.discordapp.com/attachments/781205016655560724/782253850475495444/fuuka.png")
        embed.add_field(name="ping", value="This has the bot say pong", inline=False)
        embed.add_field(name="help, h", value="Calls help", inline=False)
        embed.add_field(name="anime, a", value="Generates anime picture\n Usage: anime <optional_seed>", inline=False)
        embed.add_field(name="penis", value="Generates penis picture in your DM's\n Usage: penis <optional_seed>", inline=False)
        embed.add_field(name="furry, fu", value="Generates disgusting picture\n Usage: fu <optional_seed>", inline=False)
        embed.add_field(name="human, hu, people, ppl", value="Generates people\n Usage: human <optional_seed>", inline=False)
        embed.add_field(name="cat, c, cp", value="Generates kitty picture\n Usage: cat <optional_seed>", inline=False)
        embed.add_field(name="dog, doge, d", value="Generates doge\n Usage: dog <optional_seed>", inline=False)
        embed.add_field(name="pokemon, po", value="Generates a pokemon pic\n Usage: pokemon <optional_seed>", inline=False)
        embed.add_field(name="marry", value = "Allows to marry someone.\n Usage: marry <USER NAME>", inline=False)
        embed.add_field(name="divorce", value="Allows to divorce\n Usage: divorce", inline=False)
        #embed.add_field(name="couples", value="Shows couples\n Usage: couples", inline=False)
        embed.add_field(name="fuuka, f", value="Posts Fuuka picture\n Usage: fuuka", inline=False)
        embed.add_field(name="kiss", value="Allows to kiss someone\n Usage: kiss <USER MENTION>", inline=False)
        embed.add_field(name="fart", value = "Allows to fart on someone.\n Usage: fart <USER MENTION>", inline=False)
        embed.add_field(name="autism, au", value="Diagnosts autism in CHILDREN with 84% accuracy.\n Usage: autism <img_attachment>\n Note: for higher accuracy face should fit the entire picture.", inline=False)
        embed.add_field(name="support me", value="[Buy me a Beer](https://www.buymeacoffee.com/kosar)")
        embed.set_footer(text='Fuuka is a "Yotsubato!" manga character')
 
        await ctx.send(embed=embed)
    else:
        await ctx.send("Don'touch me, pervert!", delete_after=1)
 
 
  @bot.command(name="ping", description="pong")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def ping(ctx):
    if not in_ban(ctx.message.author):
 
      await ctx.send('Pong! `{0}ms`'.format(int(round(bot.latency, 3)*1000)))
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
  
 
  
  @bot.command(name="autism", description="Diagnosts autism", pass_context = True , aliases=['au'])
  @commands.cooldown(1, 1, commands.BucketType.user)
  async def autism(ctx, pic_url = None):
    if not in_ban(ctx.message.author):
      if ctx.message.attachments and pic_url is None:
        url = ctx.message.attachments[0].url
     
      if not ctx.message.attachments and pic_url is not None:
        url = pic_url
    
      response = requests.get(url)
      img = Image.open(BytesIO(response.content)).convert('RGB')
      #new_img = ImageOps.expand(img, border=300, fill='black')
      #img_r = img.resize((800, 600))
      img1 = img.save('/content/Autism/autistic_main.jpg')
 
    
      image = face_recognition.load_image_file("/content/Autism/autistic_main.jpg")
      face_locations = face_recognition.face_locations(image)
      if not face_locations:
        await ctx.send("Face is not detected!")
      if len(face_locations) > 5:
        face_locations = face_locations[:5]
      for index, location in enumerate(face_locations):
        top, right, bottom, left = location
        crop = (left, top, right, bottom)
        face = img.crop(crop)
        img1 = face.save('/content/Autism/autistic.jpg')
 
        transforms = T.Compose([
              T.Resize(224),
              T.CenterCrop(224),
              T.ToTensor()])
        img_tensor = transforms(face)
        img_tensor = img_tensor.to(device)

        if str(predict_image(img_tensor, Net)) == "autistic":
          pred = "autistic"
        else:
          pred = "non autistic"
        
        await ctx.send("This child is " + pred +" with 84% accuracy!")
        await ctx.send(file=discord.File('/content/Autism/autistic.jpg'))
    else:
        await ctx.send("Don'touch me, pervert!", delete_after=1)
      
 
 
  @bot.command(name="anime", description="Post generated anime picture", pass_context = True , aliases=['a'])
  @commands.cooldown(1, 8, commands.BucketType.guild)
  async def anime(ctx, seed = None):
    if not in_ban(ctx.message.author):
      await ctx.send("Generating an anime girl...", delete_after=1)
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))

      os.system('python /content/stylegan2_pytorch/run_generator.py generate_images --network=checkpoint/Gs.pth --seeds='+str(seed)+' --truncation_psi=1.0')
      await ctx.send(file=discord.File('/content/results/generated.png'))
      

    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)


  
  @bot.command(name="dog", description="Post generated dog picture", pass_context = True , aliases=['doge', 'd'])
  @commands.cooldown(1, 8, commands.BucketType.guild)
  async def dog(ctx, seed = None):
    if not in_ban(ctx.message.author):
      await ctx.send("Generating a doge...", delete_after=1)
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))

      os.system('python /content/stylegan2-ada-pytorch/generate.py --outdir=/content/out --trunc=1 --seeds='+str(seed)+' --network=/content/gdrive/MyDrive/Data/afhqdog.pkl')
      await ctx.send(file=discord.File(f'/content/out/seed{seed}.png'))
    else:
      await ctx.send("Don't touch me, pervert!", delete_after=1)

  @bot.command(name="furry", description="Post generated furry picture", pass_context = True , aliases=['fu'])
  @commands.cooldown(1, 8, commands.BucketType.guild)
  async def furry(ctx, seed = None):
    if not in_ban(ctx.message.author):
      await ctx.send("Generating a furry...", delete_after=1)
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))

      os.system('python /content/stylegan2_pytorch/run_generator.py generate_images --network=checkpoint2/Gs.pth --seeds='+str(seed)+' --truncation_psi=1.0')
      await ctx.send(file=discord.File('/content/results/generated.png'))
      

    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)

  @bot.command(name="human", description="Post generated human picture", pass_context = True , aliases=['hu', 'people', 'ppl'])
  @commands.cooldown(1, 8, commands.BucketType.guild)
  async def ppl(ctx, seed = None):
    if not in_ban(ctx.message.author):
      await ctx.send("Generating a human...", delete_after=1)
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))

      os.system('python /content/stylegan2_pytorch/run_generator.py generate_images --network=checkpoint4/Gs.pth --seeds='+str(seed)+' --truncation_psi=1.0')
      await ctx.send(file=discord.File('/content/results/generated.png'))
      

    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
    


    
  @bot.command(name="penis", description="Post generated penis picture", pass_context = True)
  @commands.cooldown(1, 8, commands.BucketType.guild)
  async def penis(ctx, seed = None):
    if not in_ban(ctx.message.author):
  
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))

      os.system('python /content/stylegan2_pytorch/run_generator.py generate_images --network=checkpoint5/Gs.pth --seeds='+str(seed)+' --truncation_psi=1.0')
      await ctx.author.send(file=discord.File('/content/results/generated.png'))
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)



  @bot.command(name="cat", description="Post generated cat picture", pass_context = True , aliases=['c',"cp"])
  @commands.cooldown(1, 8, commands.BucketType.guild)
  async def cat(ctx, seed = None):
    if not in_ban(ctx.message.author):
      await ctx.send("Generating a kitty...", delete_after=1)
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))

      os.system('python /content/stylegan2_pytorch/run_generator.py generate_images --network=checkpoint3/Gs.pth --seeds='+str(seed)+' --truncation_psi=1.0')
      await ctx.send(file=discord.File('/content/results/generated.png'))
      

    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
 
  @bot.command(name="pokemon", description="Post generated pokemon picture", pass_context = True , aliases=['po'])
  @commands.cooldown(2, 1, commands.BucketType.user)
  async def pokemon(ctx, seed = None):
    if not in_ban(ctx.message.author):
      await ctx.send("Generating a pokemon...", delete_after=1)
 
      if seed != None:
        seed = convert_to_int_2(seed)
      else:
        seed = convert_to_int_2(str(random.random()))
    
      os.system('python /content/stylegan_forpokemon/run_generator.py generate_images --network=/content/gdrive/MyDrive/Data/Gs.pth --seeds='+str(seed)+' --truncation_psi=1.0')
      await ctx.send(file=discord.File('/content/results/pokemon.png'))
      

    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
 
 
 
  
 
  @bot.command(name="kiss", description="Allows to kiss someone")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def kiss(ctx, username:discord.User=None):
    if not in_ban(ctx.message.author):
      guy = ctx.message.author
      if username is None:
          await ctx.send('{} blew a kiss'.format(guy.display_name))
          return
      if username.id == guy.id:
          await ctx.send('{} kissed ass'.format(guy.display_name))
          return
      slut = username.mention
      await ctx.send('{} kissed {}'.format( guy.display_name, slut))
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
 
  @bot.command(name="fart", description="Allows to fart on someone")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def fart(ctx, username:discord.User=None):
    if not in_ban(ctx.message.author):
      guy = ctx.message.author
    
      if username is None:
          await ctx.send('{} thought nobody would noticed this fart'.format(guy.display_name))
          return
      if username.id == guy.id:
          await ctx.send('{} farted on itself'.format(guy.display_name))
          return
 
      slut = username.mention
      await ctx.send('{} farted on {}'.format( guy.display_name, slut))
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)

 
  @bot.command(name="ban")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def ban(ctx, member:discord.User=None):
    global OWNER_ID
    if ctx.message.author.id == OWNER_ID:
      if member is None:
        await ctx.send("Nobody to ban")
        return
      guy = member.id
      if guy == ctx.message.author.id:
        await ctx.send("You cannot ban yourself!")
        return
      with open('/content/gdrive/MyDrive/Data/ban_list_rus.json') as json_file:
        data = json.load(json_file)
        #print(data)
        if guy in data['user_id']:
          await ctx.send("User is already banned")
          return
        else:
          data['user_id'].append(guy)
          await ctx.send("User added to ban list")
      with open('/content/gdrive/MyDrive/Data/ban_list_rus.json', 'w') as outfile:
        json.dump(data, outfile)
    else:
        await ctx.send("Only bot owner can use this command!", delete_after=1)
 
  def in_ban(member:discord.User=None):
    with open('/content/gdrive/MyDrive/Data/ban_list_rus.json') as json_file:
      data = json.load(json_file)
      gay = member.id
      if gay in data['user_id']:
        return True
      else:
        return False

  @bot.command(name="marry")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def marry(ctx, member:discord.User=None):
    if not in_ban(ctx.message.author):
      if member is None:
        await ctx.send("Nobody to marry")
        return
      guy = ctx.message.author.id
      if member.id == ctx.message.author.id:
        await ctx.send("You cannot marry yourself!")
        return
      with open('/content/gdrive/MyDrive/Data/marry.json') as json_file:
        data = json.load(json_file)
        print(data)
        if str(guy) in data:
          await ctx.send(f"You are already married to<@!{data[str(guy)]}>")
          return
        elif str(member.id) in data:
          await ctx.send(f"<@!{member.id}> is already married to <@!{data[str(member.id)]}>")
          return
        else:


          await ctx.send(f"<@!{member.id}>, would you like to marry <@!{guy}>?\nyes/no")  

          def check(m):
            return m.author.id == member.id 

          msg = await bot.wait_for("message", check=check)
          if msg.author.id == member.id:

            if msg.content == "Yes" or msg.content == "yes" or msg.content == "YES":

              data[str(guy)] = member.id
              data[str(member.id)] = guy
              await ctx.send(f"<@!{member.id}> and <@!{guy}> are now married!")
              with open('/content/gdrive/MyDrive/Data/marry.json', 'w') as outfile:
                json.dump(data, outfile)
              return
              

            else:
              await ctx.send(f"<@!{guy}> was rejected by <@!{member.id}>!")
              return
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)

  @bot.command(name="divorce")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def divorce(ctx):
    if not in_ban(ctx.message.author):
      guy = ctx.message.author.id
      with open('/content/gdrive/MyDrive/Data/marry.json') as json_file:
        data = json.load(json_file)
        print(data)
        if str(guy) in data:
          another = data[str(guy)]
          del data[str(guy)]
          del data[str(another)]
          if not another:
            await ctx.send(f"You don't have a partner!")
            return
          await ctx.send(f"<@!{another}> is now in divorce with <@!{guy}>")
          with open('/content/gdrive/MyDrive/Data/marry.json', 'w') as outfile:
            json.dump(data, outfile)


        else:
          await ctx.send(f"You don't have a partner!")

    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
'''
  @bot.command(name="couples")
  @commands.cooldown(4, 1, commands.BucketType.user)
  async def couples(ctx):
    if not in_ban(ctx.message.author):
      with open('/content/gdrive/MyDrive/Data/marry.json') as json_file:
        data = json.load(json_file)
        i = 1
        mess = ""
        for key, value in data.items():
          if i%2 !=0:
            user1 = bot.get_user(int(key))
            user2 = bot.get_user(int(value))
            mess += f"{user1.name} & {user2.name}\n"
          i+=1
        await ctx.send(mess)


        
    else:
      await ctx.send("Don'touch me, pervert!", delete_after=1)
'''

  @divorce.error
  @marry.error
  @kiss.error
  @fuuka.error
  @pokemon.error
  @anime.error
  @ping.error
  @fart.error
  @dog.error
  async def error(ctx, error):
    if isinstance(error, commands.CommandOnCooldown):
        msg = 'Cooldown'.format(error.retry_after)
        await ctx.send(msg, delete_after=0.5) 
    else:
        raise error
    
 
 
    

 


In [ ]:
bot.run(TOKEN)